# How to Inspect Function and Class Signatures in Python

[![Twitter Handle](https://img.shields.io/badge/Twitter-@gaohongnan-blue?style=social&logo=twitter)](https://twitter.com/gaohongnan)
[![LinkedIn Profile](https://img.shields.io/badge/@gaohongnan-blue?style=social&logo=linkedin)](https://linkedin.com/in/gao-hongnan)
[![GitHub Profile](https://img.shields.io/badge/GitHub-gao--hongnan-lightgrey?style=social&logo=github)](https://github.com/gao-hongnan)
[![Code](https://img.shields.io/badge/View-Code-blue?style=flat-square&logo=github)](https://github.com/gao-hongnan/omniverse/blob/main/omnivault/utils/inspector/core.py)
![Tag](https://img.shields.io/badge/Tag-Brain_Dump-red)


```{contents}
:local:
```

In [74]:
%pip install -U -q langchain

Note: you may need to restart the kernel to use updated packages.


In [75]:
from transformers import Trainer, GPT2LMHeadModel, TrainingArguments
from dataclasses import field, make_dataclass

import inspect
from inspect import Signature, Parameter
from typing import Any, Callable, Dict, Set, Optional, _GenericAlias, List, Type, Union, get_type_hints, overload, Tuple
from pydantic import BaseModel
from rich.pretty import pprint

- Don't need to use `getfullargspec` as it seems to be to be more of an old remnant of the `getargspec` method. See [here](https://github.com/joblib/joblib/issues/1164).
- how about if the class is subclass of another, so u need to get all arg of original class too

## Construct Hypothetical Function, Child and Parent Classes

In [76]:
class ParentClass:
    """This is the parent class."""

    parent_class_attr = 'a parent class attribute'

    def __init__(self, parent_instance_attr: str) -> None:
        self.parent_instance_attr = parent_instance_attr

    def parent_method(self) -> str:
        """This is a method in the parent class."""
        return "Parent method called"

class ChildClass(ParentClass):
    """This is a subclass of ParentClass."""

    # Class attribute
    class_attr = 'a class attribute'

    # Private and protected attributes
    _protected_attr = 'a protected attribute'
    __private_attr = 'a private attribute'

    def __init__(self, instance_attr: str, parent_instance_attr: str) -> None:
        """Initialize the instance."""
        super().__init__(parent_instance_attr)
        # Instance attribute
        self.instance_attr = instance_attr

    @property
    def read_only_attr(self) -> str:
        """This is a read-only attribute."""
        return 'You can read me, but you cannot change me.'

    def instance_method(self, arg: str) -> str:
        """This is an instance method."""
        return f'Instance method called with argument: {arg}'

    @classmethod
    def class_method(cls, arg: str) -> str:
        """This is a class method."""
        return f'Class method called with argument: {arg}'

    @staticmethod
    def static_method(arg: str) -> str:
        """This is a static method."""
        return f'Static method called with argument: {arg}'

    def __str__(self) -> str:
        """Return a string representation of the instance."""
        return f'MyClass(instance_attr={self.instance_attr})'


In [77]:
instance_child = ChildClass(instance_attr='an instance attribute', parent_instance_attr='a parent instance attribute')
class_child = ChildClass

instance_parent = ParentClass(parent_instance_attr='a parent instance attribute')
class_parent = ParentClass

In [78]:
def func(a: int, b: str, c: List[int], d: Tuple[str, str], e: Union[int, str], **kwargs: Any) -> str:
    return a, b, c, d, e, kwargs

## Inspect All Members

In [79]:
@overload
def get_members_of_function_or_method(
    func_or_class: Type[object], predicate: Optional[Callable[[Any], bool]] = None
) -> List[Tuple[str, Any]]:
    ...


@overload
def get_members_of_function_or_method(
    func_or_class: Callable[..., Any], predicate: Optional[Callable[[Any], bool]] = None
) -> List[Tuple[str, Any]]:
    ...


def get_members_of_function_or_method(
    func_or_class: Union[Type[object], Callable[..., Any]], predicate: Optional[Callable[[Any], bool]] = None
) -> List[Tuple[str, Any]]:
    return inspect.getmembers(func_or_class, predicate)

def loop_through_members(members: List[Tuple[str, Any]], filter: Optional[str] = None) -> None:
    if filter is not None:
        members = [member for member in members if filter in member[0]]
    for member in members:
        name, value = member
        print(f'{name}: {value}')

Our initial goal is to get all signatures and type annotations of a class or function. We can use the `inspect` module to achieve this. The `getmembers` function returns all members of a class or module. We can then filter out the functions and classes and inspect their signatures.

However, for our purpose, it may be overkill since it retrieves all members
within a module, the scope is very broad, for example, inspecting just the `func`
defined will also return all `__globals__`, which may not be what we want.

In [80]:
func_all_members = get_members_of_function_or_method(func, predicate=None)
loop_through_members(func_all_members)

__annotations__: {'a': <class 'int'>, 'b': <class 'str'>, 'c': typing.List[int], 'd': typing.Tuple[str, str], 'e': typing.Union[int, str], 'kwargs': typing.Any, 'return': <class 'str'>}
__call__: <method-wrapper '__call__' of function object at 0x2a7d5b310>
__class__: <class 'function'>
__closure__: None
__code__: <code object func at 0x2a9741be0, file "/var/folders/l2/jjqj299126j0gycr9kkkt9xm0000gn/T/ipykernel_16129/2139551385.py", line 1>
__defaults__: None
__delattr__: <method-wrapper '__delattr__' of function object at 0x2a7d5b310>
__dict__: {}
__dir__: <built-in method __dir__ of function object at 0x2a7d5b310>
__doc__: None
__eq__: <method-wrapper '__eq__' of function object at 0x2a7d5b310>
__format__: <built-in method __format__ of function object at 0x2a7d5b310>
__ge__: <method-wrapper '__ge__' of function object at 0x2a7d5b310>
__get__: <method-wrapper '__get__' of function object at 0x2a7d5b310>
__getattribute__: <method-wrapper '__getattribute__' of function object at 0x2a7d

And to get the signature, we can just filter `'__annotations__'`.

In [81]:
loop_through_members(func_all_members, filter='__annotations__')

__annotations__: {'a': <class 'int'>, 'b': <class 'str'>, 'c': typing.List[int], 'd': typing.Tuple[str, str], 'e': typing.Union[int, str], 'kwargs': typing.Any, 'return': <class 'str'>}


In [82]:
class_child_all_members = get_members_of_function_or_method(class_child, predicate=None)
loop_through_members(class_child_all_members)

_ChildClass__private_attr: a private attribute
__class__: <class 'type'>
__delattr__: <slot wrapper '__delattr__' of 'object' objects>
__dict__: {'__module__': '__main__', '__doc__': 'This is a subclass of ParentClass.', 'class_attr': 'a class attribute', '_protected_attr': 'a protected attribute', '_ChildClass__private_attr': 'a private attribute', '__init__': <function ChildClass.__init__ at 0x2a95d9670>, 'read_only_attr': <property object at 0x2a9451590>, 'instance_method': <function ChildClass.instance_method at 0x2a95d9430>, 'class_method': <classmethod object at 0x2a957aa00>, 'static_method': <staticmethod object at 0x2a957abe0>, '__str__': <function ChildClass.__str__ at 0x2a95d9700>}
__dir__: <method '__dir__' of 'object' objects>
__doc__: This is a subclass of ParentClass.
__eq__: <slot wrapper '__eq__' of 'object' objects>
__format__: <method '__format__' of 'object' objects>
__ge__: <slot wrapper '__ge__' of 'object' objects>
__getattribute__: <slot wrapper '__getattribute__

In [83]:
instance_child_all_members = get_members_of_function_or_method(instance_child, predicate=None)
loop_through_members(instance_child_all_members)

_ChildClass__private_attr: a private attribute
__class__: <class '__main__.ChildClass'>
__delattr__: <method-wrapper '__delattr__' of ChildClass object at 0x2a947ff40>
__dict__: {'parent_instance_attr': 'a parent instance attribute', 'instance_attr': 'an instance attribute'}
__dir__: <built-in method __dir__ of ChildClass object at 0x2a947ff40>
__doc__: This is a subclass of ParentClass.
__eq__: <method-wrapper '__eq__' of ChildClass object at 0x2a947ff40>
__format__: <built-in method __format__ of ChildClass object at 0x2a947ff40>
__ge__: <method-wrapper '__ge__' of ChildClass object at 0x2a947ff40>
__getattribute__: <method-wrapper '__getattribute__' of ChildClass object at 0x2a947ff40>
__gt__: <method-wrapper '__gt__' of ChildClass object at 0x2a947ff40>
__hash__: <method-wrapper '__hash__' of ChildClass object at 0x2a947ff40>
__init__: <bound method ChildClass.__init__ of <__main__.ChildClass object at 0x2a947ff40>>
__init_subclass__: <built-in method __init_subclass__ of type obje

In [84]:
trainer_all_members = get_members_of_function_or_method(Trainer, predicate=None)
loop_through_members(trainer_all_members)

__class__: <class 'type'>
__delattr__: <slot wrapper '__delattr__' of 'object' objects>
__dict__: {'__module__': 'transformers.trainer', '__doc__': "\n    Trainer is a simple but feature-complete training and eval loop for PyTorch, optimized for 🤗 Transformers.\n\n    Args:\n        model ([`PreTrainedModel`] or `torch.nn.Module`, *optional*):\n            The model to train, evaluate or use for predictions. If not provided, a `model_init` must be passed.\n\n            <Tip>\n\n            [`Trainer`] is optimized to work with the [`PreTrainedModel`] provided by the library. You can still use\n            your own models defined as `torch.nn.Module` as long as they work the same way as the 🤗 Transformers\n            models.\n\n            </Tip>\n\n        args ([`TrainingArguments`], *optional*):\n            The arguments to tweak for training. Will default to a basic instance of [`TrainingArguments`] with the\n            `output_dir` set to a directory named *tmp_trainer* in the 

## Retrieve All Methods of a Class

There are a few ways to do it.

### Using `__dict__`

In [85]:
child_class_methods_using_dict = list(ChildClass.__dict__.keys())
pprint(sorted(child_class_methods_using_dict))

assert 'parent_method' not in child_class_methods_using_dict
assert 'read_only_attr' in child_class_methods_using_dict
assert 'class_method' in child_class_methods_using_dict

[
│   '_ChildClass__private_attr',
│   '__doc__',
│   '__init__',
│   '__module__',
│   '__str__',
│   '_protected_attr',
│   'class_attr',
│   'class_method',
│   'instance_method',
│   'read_only_attr',
│   'static_method'
]

Notice that the parent class methods are not included!

In [86]:
pprint(instance_child.__class__.__dict__.keys() == ChildClass.__dict__.keys())

True

### Using `vars`

`vars` and `__dict__` are equivalent, but people are preferring the former due
to some efficiency reasons, which can be found [in this post](https://stackoverflow.com/questions/21297203/use-dict-or-vars).

In [87]:
child_class_methods_using_vars = list(vars(ChildClass).keys())
pprint(sorted(child_class_methods_using_vars))

assert 'parent_method' not in child_class_methods_using_vars
assert 'read_only_attr' in child_class_methods_using_vars
assert 'class_method' in child_class_methods_using_vars

assert set(child_class_methods_using_dict) == set(child_class_methods_using_vars)

[
│   '_ChildClass__private_attr',
│   '__doc__',
│   '__init__',
│   '__module__',
│   '__str__',
│   '_protected_attr',
│   'class_attr',
│   'class_method',
│   'instance_method',
│   'read_only_attr',
│   'static_method'
]

### Using `dir`

To include the base/parent class methods, we can use `dir` instead.

In [88]:
child_class_methods_using_dir = list(dir(ChildClass))
pprint(sorted(child_class_methods_using_dir))

assert 'parent_method' in child_class_methods_using_dir
assert 'read_only_attr' in child_class_methods_using_dir
assert 'class_method' in child_class_methods_using_dir

[
│   '_ChildClass__private_attr',
│   '__class__',
│   '__delattr__',
│   '__dict__',
│   '__dir__',
│   '__doc__',
│   '__eq__',
│   '__format__',
│   '__ge__',
│   '__getattribute__',
│   '__gt__',
│   '__hash__',
│   '__init__',
│   '__init_subclass__',
│   '__le__',
│   '__lt__',
│   '__module__',
│   '__ne__',
│   '__new__',
│   '__reduce__',
│   '__reduce_ex__',
│   '__repr__',
│   '__setattr__',
│   '__sizeof__',
│   '__str__',
│   '__subclasshook__',
│   '__weakref__',
│   '_protected_attr',
│   'class_attr',
│   'class_method',
│   'instance_method',
│   'parent_class_attr',
│   'parent_method',
│   'read_only_attr',
│   'static_method'
]

### Using `inspect.getmembers`

We use `inspect.getmembers` to get all members of a class, and then filter out
via the predicate `inspect.isroutine`, a stronger filter than `inspect.isfunction`
or `inspect.ismethod`.

We attach the source code of `inspect.isroutine` here for reference.

```python
def isroutine(object):
    """Return true if the object is any kind of function or method."""
    return (isbuiltin(object)
            or isfunction(object)
            or ismethod(object)
            or ismethoddescriptor(object))
```

In [89]:
predicate = inspect.isroutine
child_class_methods_using_getmembers = list(get_members_of_function_or_method(ChildClass, predicate=predicate))

pprint(sorted(child_class_methods_using_getmembers))

[
│   ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
│   ('__dir__', <method '__dir__' of 'object' objects>),
│   ('__eq__', <slot wrapper '__eq__' of 'object' objects>),
│   ('__format__', <method '__format__' of 'object' objects>),
│   ('__ge__', <slot wrapper '__ge__' of 'object' objects>),
│   ('__getattribute__', <slot wrapper '__getattribute__' of 'object' objects>),
│   ('__gt__', <slot wrapper '__gt__' of 'object' objects>),
│   ('__hash__', <slot wrapper '__hash__' of 'object' objects>),
│   ('__init__', <function ChildClass.__init__ at 0x2a95d9670>),
│   ('__init_subclass__', <built-in method __init_subclass__ of type object at 0x11e950d80>),
│   ('__le__', <slot wrapper '__le__' of 'object' objects>),
│   ('__lt__', <slot wrapper '__lt__' of 'object' objects>),
│   ('__ne__', <slot wrapper '__ne__' of 'object' objects>),
│   ('__new__', <built-in method __new__ of type object at 0x100bc6b00>),
│   ('__reduce__', <method '__reduce__' of 'object' objects>),
│   ('__reduce_ex__', <method '__reduce_ex__' of 'object' objects>),
│   ('__repr__', <slot wrapper '__repr__' of 'object' objects>),
│   ('__setattr__', <slot wrapper '__setattr__' of 'object' objects>),
│   ('__sizeof__', <method '__sizeof__' of 'object' objects>),
│   ('__str__', <function ChildClass.__str__ at 0x2a95d9700>),
│   ('__subclasshook__', <built-in method __subclasshook__ of type object at 0x11e950d80>),
│   ('class_method', <bound method ChildClass.class_method of <class '__main__.ChildClass'>>),
│   ('instance_method', <function ChildClass.instance_method at 0x2a95d9430>),
│   ('parent_method', <function ParentClass.parent_method at 0x2a95d93a0>),
│   ('static_method', <function ChildClass.static_method at 0x2a95d9a60>)
]

Of course, the reason to retrieve all methods is a convenience if we want to
inspect all methods at once. And if we can obtain all methods, we can then
iteratively inspect each method's signature.

### Method Resolution Order

The above examples do not take into account complicated cases, such as when
the class is a subclass of **multiple** classes, in which case if you just
print out the methods of the class, you will have a hard time to know which
methods are from which class. You can do so via more filtering, but this is
beyond the scope of this notebook. 

In [90]:
predicate = inspect.isroutine
GPT2LMHeadModel_methods_using_getmembers = list(get_members_of_function_or_method(GPT2LMHeadModel, predicate=predicate))

pprint(sorted(GPT2LMHeadModel_methods_using_getmembers))

[
│   ('__call__', <function Module._wrapped_call_impl at 0x11cdae1f0>),
│   ('__delattr__', <function Module.__delattr__ at 0x11cdae550>),
│   ('__dir__', <function Module.__dir__ at 0x11cdb2430>),
│   ('__eq__', <slot wrapper '__eq__' of 'object' objects>),
│   ('__format__', <method '__format__' of 'object' objects>),
│   ('__ge__', <slot wrapper '__ge__' of 'object' objects>),
│   ('__getattr__', <function Module.__getattr__ at 0x11cdae430>),
│   ('__getattribute__', <slot wrapper '__getattribute__' of 'object' objects>),
│   ('__getstate__', <function Module.__getstate__ at 0x11cdae310>),
│   ('__gt__', <slot wrapper '__gt__' of 'object' objects>),
│   ('__hash__', <slot wrapper '__hash__' of 'object' objects>),
│   ('__init__', <function GPT2LMHeadModel.__init__ at 0x2a7b925e0>),
│   ('__init_subclass__', <built-in method __init_subclass__ of type object at 0x2a3e3a9d0>),
│   ('__le__', <slot wrapper '__le__' of 'object' objects>),
│   ('__lt__', <slot wrapper '__lt__' of 'object' objects>),
│   ('__ne__', <slot wrapper '__ne__' of 'object' objects>),
│   ('__new__', <built-in method __new__ of type object at 0x100bc6b00>),
│   ('__reduce__', <method '__reduce__' of 'object' objects>),
│   ('__reduce_ex__', <method '__reduce_ex__' of 'object' objects>),
│   ('__repr__', <function Module.__repr__ at 0x11cdb23a0>),
│   ('__setattr__', <function Module.__setattr__ at 0x11cdae4c0>),
│   ('__setstate__', <function Module.__setstate__ at 0x11cdae3a0>),
│   ('__sizeof__', <method '__sizeof__' of 'object' objects>),
│   ('__str__', <slot wrapper '__str__' of 'object' objects>),
│   ('__subclasshook__', <built-in method __subclasshook__ of type object at 0x2a3e3a9d0>),
│   ('_apply', <function Module._apply at 0x11cdab550>),
│   (
│   │   '_autoset_attn_implementation',
│   │   <bound method PreTrainedModel._autoset_attn_implementation of <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>>
│   ),
│   (
│   │   '_backward_compatibility_gradient_checkpointing',
│   │   <function PreTrainedModel._backward_compatibility_gradient_checkpointing at 0x2a72b3670>
│   ),
│   ('_call_impl', <function Module._call_impl at 0x11cdae280>),
│   (
│   │   '_check_and_enable_flash_attn_2',
│   │   <bound method PreTrainedModel._check_and_enable_flash_attn_2 of <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>>
│   ),
│   (
│   │   '_check_and_enable_sdpa',
│   │   <bound method PreTrainedModel._check_and_enable_sdpa of <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>>
│   ),
│   ('_convert_head_mask_to_5d', <function ModuleUtilsMixin._convert_head_mask_to_5d at 0x2a72b31f0>),
│   (
│   │   '_copy_lm_head_original_to_resized',
│   │   <function PreTrainedModel._copy_lm_head_original_to_resized at 0x2a72b6430>
│   ),
│   ('_create_repo', <function PushToHubMixin._create_repo at 0x12cb2a9d0>),
│   ('_dispatch_accelerate_model', <function PeftAdapterMixin._dispatch_accelerate_model at 0x2a7279e50>),
│   ('_expand_inputs_for_generation', <function GenerationMixin._expand_inputs_for_generation at 0x2a727e430>),
│   (
│   │   '_extract_past_from_model_output',
│   │   <function GenerationMixin._extract_past_from_model_output at 0x2a727e4c0>
│   ),
│   (
│   │   '_from_config',
│   │   <bound method PreTrainedModel._from_config of <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>>
│   ),
│   ('_get_backward_hooks', <function Module._get_backward_hooks at 0x11cdabe50>),
│   ('_get_backward_pre_hooks', <function Module._get_backward_pre_hooks at 0x11cdabee0>),
│   ('_get_candidate_generator', <function GenerationMixin._get_candidate_generator at 0x2a727e670>),
│   ('_get_decoder_start_token_id', <function GenerationMixin._get_decoder_start_token_id at 0x2a727e3a0>),
│   ('_get_files_timestamps', <function PushToHubMixin._get_files_timestamps at 0x12cb2aa60>),
│   ('_get_generation_mode', <function GenerationMixin._get_generation_mode at 0x2a727e790>),
│   ('_get_logits_processor', <function Generat

You can get the method resolution order (MRO) of a class via `cls.__mro__`. 

In [91]:
inspect.getmro(GPT2LMHeadModel)

(transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel,
 transformers.models.gpt2.modeling_gpt2.GPT2PreTrainedModel,
 transformers.modeling_utils.PreTrainedModel,
 torch.nn.modules.module.Module,
 transformers.modeling_utils.ModuleUtilsMixin,
 transformers.generation.utils.GenerationMixin,
 transformers.utils.hub.PushToHubMixin,
 transformers.integrations.peft.PeftAdapterMixin,
 object)

A pseudocode to get all signatures of a class via MRO is as follows:

```python
def get_all_args(cls: Type[object]) -> Dict[str, inspect.Parameter]:
    mro = inspect.getmro(cls)
    all_args = {}
    for base_class in mro[::-1]:  # reverse to start from topmost class
        if base_class is object:  # skip the base 'object' class
            continue
        sig = inspect.signature(base_class.__init__)
        all_args.update(sig.parameters)
    return all_args
```


## Get Signature and Type Annotations of a Function

In [92]:
func_sig: Signature = inspect.signature(func)
pprint(func_sig.parameters)
pprint(func_sig.return_annotation)

mappingproxy({
│   'a': <Parameter "a: int">,
│   'b': <Parameter "b: str">,
│   'c': <Parameter "c: List[int]">,
│   'd': <Parameter "d: Tuple[str, str]">,
│   'e': <Parameter "e: Union[int, str]">,
│   'kwargs': <Parameter "**kwargs: Any">
})

<class 'str'>

Here are the 4 key properties of the `Parameter` object
of the `Signature` object.

```python
@property
def name(self):
    return self._name

@property
def default(self):
    return self._default

@property
def annotation(self):
    return self._annotation

@property
def kind(self):
    return self._kind
```

We will also use `get_type_hints` to get the type hints of a function
instead of using the `annotations` property of `inspect.Signature`. The reason
can be found in the docstring of `get_type_hints`:

```python
def get_type_hints(obj, globalns=None, localns=None, include_extras=False):
    """Return type hints for an object.

    This is often the same as obj.__annotations__, but it handles
    forward references encoded as string literals, adds Optional[t] if a
    default value equal to None is set and recursively replaces all
    'Annotated[T, ...]' with 'T' (unless 'include_extras=True').

    ...
    """
```

In [93]:
def no_type_hints(a, b, c, d, e, **kwargs):
    return a, b, c, d, e, kwargs

In [94]:
get_type_hints(no_type_hints), inspect.signature(no_type_hints)

({}, <Signature (a, b, c, d, e, **kwargs)>)

How to know if a parameter is optional or not? We can use the `inspect.Parameter.empty`
property.

In [95]:
for name, value in inspect.signature(func).parameters.items():
    print(value.default)
    print(value.default is inspect.Parameter.empty)

<class 'inspect._empty'>
True
<class 'inspect._empty'>
True
<class 'inspect._empty'>
True
<class 'inspect._empty'>
True
<class 'inspect._empty'>
True
<class 'inspect._empty'>
True


We will also use `__mro__` to get the method resolution order of a class
because `__bases__` only returns the immediate parent class.

In [96]:
ChildClass.__bases__, GPT2LMHeadModel.__bases__[0].__bases__[0].__bases__

((__main__.ParentClass,),
 (torch.nn.modules.module.Module,
  transformers.modeling_utils.ModuleUtilsMixin,
  transformers.generation.utils.GenerationMixin,
  transformers.utils.hub.PushToHubMixin,
  transformers.integrations.peft.PeftAdapterMixin))

In [97]:
list(reversed(inspect.getmro(GPT2LMHeadModel)))

[object,
 transformers.integrations.peft.PeftAdapterMixin,
 transformers.utils.hub.PushToHubMixin,
 transformers.generation.utils.GenerationMixin,
 transformers.modeling_utils.ModuleUtilsMixin,
 torch.nn.modules.module.Module,
 transformers.modeling_utils.PreTrainedModel,
 transformers.models.gpt2.modeling_gpt2.GPT2PreTrainedModel,
 transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel]

In [98]:
def get_base_classes(cls: Type[Any], include_self: bool = False) -> Set[Type[Any]]:
    """
    Get the base classes of a class and all its base classes.
    """
    return set(cls.__mro__[0:-1] if include_self else cls.__mro__[1:-1])

pprint(get_base_classes(GPT2LMHeadModel, include_self=True))

{
│   <class 'transformers.modeling_utils.ModuleUtilsMixin'>,
│   <class 'transformers.utils.hub.PushToHubMixin'>,
│   <class 'transformers.models.gpt2.modeling_gpt2.GPT2PreTrainedModel'>,
│   <class 'transformers.generation.utils.GenerationMixin'>,
│   <class 'torch.nn.modules.module.Module'>,
│   <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>,
│   <class 'transformers.integrations.peft.PeftAdapterMixin'>,
│   <class 'transformers.modeling_utils.PreTrainedModel'>
}

In [99]:
def get_default(param: Parameter) -> Any:
    """Return the parameter's default value or None if not specified."""
    return param.default if param.default is not param.empty else None

def get_field_annotations(func_or_method: Callable[..., Any]) -> Tuple[List[Tuple[str, Any, Any]], Dict[str, Any]]:
    if not inspect.isroutine(func_or_method):
        raise ValueError("Expected a function or method")

    required_fields = []
    optional_fields = []
    annotations = {}

    try:
        sig: Signature = inspect.signature(func_or_method)
        type_hints: Dict[str, Any] = get_type_hints(func_or_method)
    except ValueError:
        raise ValueError("Object does not support signature or type hints extraction.") from None

    for name, param in sig.parameters.items():
        if name == "self":
            continue

        type_hint = type_hints.get(name, Any)
        annotations[name] = type_hint
        if param.default is param.empty:
            required_fields.append((name, type_hint, Ellipsis))
        else:
            default_value = param.default
            optional_fields.append((name, type_hint, default_value))

    fields = required_fields + optional_fields
    return fields, annotations


# TODO: Tuple[str, Any, Any] should be Tuple[str, Any, ellipsis]
def get_constructor_field_annotations(
    cls: Type[Any], include_bases: bool = True
) -> Tuple[List[Tuple[str, Any, Any]], Dict[str, Any]]:
    fields = []
    annotations = {}

    classes_to_inspect = [cls] + list(get_base_classes(cls, include_self=False)) if include_bases else [cls]

    for c in reversed(classes_to_inspect):  # Reverse to respect MRO
        if hasattr(c, "__init__"):
            class_fields, class_annotations = get_field_annotations(c.__init__)
            # Update fields and annotations with those from the current class,
            # avoiding duplicates.
            for field in class_fields:
                if field[0] not in annotations:
                    fields.append(field)  # noqa: PERF401
            annotations.update(class_annotations)

    return fields, annotations

In [112]:
fields, annotations = get_constructor_field_annotations(TrainingArguments, include_bases=False)
for field in fields:
    assert len(field) == 3
    print(f"{field[0]}, {field[1]}, {field[2]}")

assert get_field_annotations(TrainingArguments.__init__) == (fields, annotations)

output_dir, <class 'str'>, Ellipsis
overwrite_output_dir, <class 'bool'>, False
do_train, <class 'bool'>, False
do_eval, <class 'bool'>, False
do_predict, <class 'bool'>, False
evaluation_strategy, typing.Union[transformers.trainer_utils.IntervalStrategy, str], no
prediction_loss_only, <class 'bool'>, False
per_device_train_batch_size, <class 'int'>, 8
per_device_eval_batch_size, <class 'int'>, 8
per_gpu_train_batch_size, typing.Optional[int], None
per_gpu_eval_batch_size, typing.Optional[int], None
gradient_accumulation_steps, <class 'int'>, 1
eval_accumulation_steps, typing.Optional[int], None
eval_delay, typing.Optional[float], 0
learning_rate, <class 'float'>, 5e-05
weight_decay, <class 'float'>, 0.0
adam_beta1, <class 'float'>, 0.9
adam_beta2, <class 'float'>, 0.999
adam_epsilon, <class 'float'>, 1e-08
max_grad_norm, <class 'float'>, 1.0
num_train_epochs, <class 'float'>, 3.0
max_steps, <class 'int'>, -1
lr_scheduler_type, typing.Union[transformers.trainer_utils.SchedulerType, str

Warning: it does not play too well with `dataclass` and `pydantic` classes
because they have more complex bells and whistles. However, because of the perks
of `dataclass` and `pydantic`, we can just use
[property](https://stackoverflow.com/questions/71183960/short-way-to-get-all-field-names-of-a-pydantic-class)
like `model_fields` to get all fields and their types.

As we can see from above, we did not handle `lr_scheduler_kwargs` well:

```python
lr_scheduler_kwargs, typing.Optional[typing.Dict], <factory>
```

where `<factory>` is the default value of the parameter. But it is actually
referring to the `default_factory` of the `dataclass` field, which can be a default
dict etc.

In [114]:
def type_hint_to_str(type_hint: Any) -> str:
    """
    Convert a type hint into its string representation.
    """
    if hasattr(type_hint, '__name__'):
        return type_hint.__name__
    elif hasattr(type_hint, '_name') and type_hint._name is not None:
        return str(type_hint._name)
    elif type(type_hint) == _GenericAlias:  # For Python 3.8+
        # Handles complex types, e.g., List[int], Union[str, int]
        origin = type_hint_to_str(type_hint.__origin__)
        args = ', '.join(type_hint_to_str(arg) for arg in type_hint.__args__)
        return f"{origin}[{args}]"
    else:
        # Fallback for unhandled types
        return str(type_hint)

def create_config_class_str(fields: List[Tuple[str, Any, Any]]) -> str:
    lines = ["class Config:"]
    if not fields:
        lines.append("    ...")
    else:
        init_params = ["self"]
        init_body = []
        for name, type_hint, default in fields:
            type_hint_str = type_hint_to_str(type_hint)
            if default is Ellipsis:  # Required argument
                param_str = f"{name}: {type_hint_str}"
            elif default is field:
                param_str = f"{name}: {type_hint_str} = field(default_factory=dict)"
            else:
                default_repr = repr(default) if default is not None else 'None'
                param_str = f"{name}: {type_hint_str} = {default_repr}"

            init_params.append(param_str)
            init_body.append(f"        self.{name} = {name}")

        lines.append(f"    def __init__({', '.join(init_params)}):")
        lines.extend(init_body)

    return '\n'.join(lines)

config_class_str = create_config_class_str(fields)
print(config_class_str)

class Config:
    def __init__(self, output_dir: str, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, evaluation_strategy: typing.Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: typing.Optional[int] = None, per_gpu_eval_batch_size: typing.Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: typing.Optional[int] = None, eval_delay: typing.Optional[float] = 0, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: typing.Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: typing.Optional[typing.Dict] = <factory>, warmup_ratio: float =

Using this as is will yield a `SyntaxError` because of the `<factory>` issue
highlighted above. We can use on a "normal" class `Trainer`.

In [117]:
fields, annotations = get_constructor_field_annotations(Trainer, include_bases=False)
config_class_str = create_config_class_str(fields)
print(config_class_str)

class Config:
    def __init__(self, model: typing.Union[transformers.modeling_utils.PreTrainedModel, torch.nn.modules.module.Module, NoneType] = None, args: typing.Optional[transformers.training_args.TrainingArguments] = None, data_collator: typing.Optional[DataCollator] = None, train_dataset: typing.Optional[torch.utils.data.dataset.Dataset] = None, eval_dataset: typing.Union[torch.utils.data.dataset.Dataset, typing.Dict[str, torch.utils.data.dataset.Dataset], NoneType] = None, tokenizer: typing.Optional[transformers.tokenization_utils_base.PreTrainedTokenizerBase] = None, model_init: typing.Optional[typing.Callable[[], transformers.modeling_utils.PreTrainedModel]] = None, compute_metrics: typing.Optional[typing.Callable[[transformers.trainer_utils.EvalPrediction], typing.Dict]] = None, callbacks: typing.Optional[typing.List[transformers.trainer_callback.TrainerCallback]] = None, optimizers: Tuple = (None, None), preprocess_logits_for_metrics: typing.Optional[typing.Callable[[torch.T

In [124]:
import transformers
import typing
import torch
from transformers import DataCollator

NoneType = type(None)

config_class_str = create_config_class_str(fields)

# Execute the generated class definition string
namespace = {}
exec(config_class_str, globals(), namespace)

# Extract the newly created class from the namespace
ConfigClass = namespace['Config']


In [126]:
inspect.signature(ConfigClass.__init__)

<Signature (self, model: Union[transformers.modeling_utils.PreTrainedModel, torch.nn.modules.module.Module, NoneType] = None, args: Optional[transformers.training_args.TrainingArguments] = None, data_collator: Optional[DataCollator] = None, train_dataset: Optional[torch.utils.data.dataset.Dataset] = None, eval_dataset: Union[torch.utils.data.dataset.Dataset, Dict[str, torch.utils.data.dataset.Dataset], NoneType] = None, tokenizer: Optional[transformers.tokenization_utils_base.PreTrainedTokenizerBase] = None, model_init: Optional[Callable[[], transformers.modeling_utils.PreTrainedModel]] = None, compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], Dict]] = None, callbacks: Optional[List[transformers.trainer_callback.TrainerCallback]] = None, optimizers: Tuple = (None, None), preprocess_logits_for_metrics: Optional[Callable[[torch.Tensor, torch.Tensor], torch.Tensor]] = None)>

## References and Further Readings

- [inspect — Inspect live objects](https://docs.python.org/3/library/inspect.html)
- [Getting attributes of a class](https://stackoverflow.com/questions/9058305/getting-attributes-of-a-class)
- [Use __dict__ or vars()?](https://stackoverflow.com/questions/21297203/use-dict-or-vars)
- [How do I get list of methods in a Python class?](https://stackoverflow.com/questions/1911281/how-do-i-get-list-of-methods-in-a-python-class)